Load the optimized model by executing the cell below

In [2]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('drive_road_following_model_trt.pth'))

OSError: /usr/lib/aarch64-linux-gnu/libgomp.so.1: cannot allocate memory in static TLS block

Create the racecar class

In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()
car.throttle_gain = 0.4

Create the camera class.

In [7]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=10)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [17]:
from utils import preprocess

def funStuff():
    try:
        while True:
            image = camera.read()
            image = preprocess(image).half()
            output = model_trt(image).detach().cpu().numpy().flatten()
            car.steering = float(output[0])
            car.throttle = float(output[1])
    finally:
        car.throttle = 0.0
        car.steering = 0.0

KeyboardInterrupt: 

Now run the function to make the car drive. Because it is a function, when running the code from the command line, rather than Jupyter notebook, it is easy to kill it with ctrl+c to stop the car, and restart calling the function again.

In [ ]:
funStuff()